<a href="https://colab.research.google.com/github/stuti-go/superhero-name-generator/blob/main/Superhero_Name_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Superhero (and Supervillain) Name Generator

---

[Superhero Names Dataset](https://github.com/am1tyadav/superhero)

## Task 2

1. Import the data
2. Create a tokenizer
3. Char to index and Index to char dictionaries

In [4]:
!git clone https://github.com/am1tyadav/superhero

Cloning into 'superhero'...
remote: Enumerating objects: 8, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 8 (delta 0), reused 4 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (8/8), 47.08 KiB | 395.00 KiB/s, done.


In [5]:
with open('superhero/superheroes.txt', 'r') as f:
  data=f.read()

data[:100]

'jumpa\t\ndoctor fate\t\nstarlight\t\nisildur\t\nlasher\t\nvarvara\t\nthe target\t\naxel\t\nbattra\t\nchangeling\t\npyrrh'

In [6]:
import tensorflow as tf

In [7]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(
    filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~',
    split='\n',
)

In [8]:
tokenizer.fit_on_texts(data)

In [9]:
char_to_index = tokenizer.word_index
index_to_char = dict((v,k) for k,v in char_to_index.items())

print(index_to_char)

{1: '\t', 2: 'a', 3: 'e', 4: 'r', 5: 'o', 6: 'n', 7: 'i', 8: ' ', 9: 't', 10: 's', 11: 'l', 12: 'm', 13: 'h', 14: 'd', 15: 'c', 16: 'u', 17: 'g', 18: 'k', 19: 'b', 20: 'p', 21: 'y', 22: 'w', 23: 'f', 24: 'v', 25: 'j', 26: 'z', 27: 'x', 28: 'q'}


## Task 3

1. Converting between names and sequences

In [10]:
names = data.splitlines()

In [11]:
tokenizer.texts_to_sequences(names[0])

[[25], [16], [12], [20], [2], [1]]

In [12]:
def name_to_seq(name):
  return [tokenizer.texts_to_sequences(c)[0][0] for c in name]

In [13]:
name_to_seq(names[0])

[25, 16, 12, 20, 2, 1]

In [14]:
def seq_to_name(seq):
  return ''.join([index_to_char[i] for i in seq if i !=0])

In [15]:
seq_to_name([25, 16, 12, 20, 2, 1])

'jumpa\t'

## Task 4

1. Creating sequences
2. Padding all sequences

In [16]:
sequences = []

for name in names:
  seq= name_to_seq(name)
  if len(seq) >= 2:
    sequences += [seq[:i] for i in range(2, len(seq)+1)]

In [17]:
sequences[:10]

[[25, 16],
 [25, 16, 12],
 [25, 16, 12, 20],
 [25, 16, 12, 20, 2],
 [25, 16, 12, 20, 2, 1],
 [14, 5],
 [14, 5, 15],
 [14, 5, 15, 9],
 [14, 5, 15, 9, 5],
 [14, 5, 15, 9, 5, 4]]

In [18]:
max_len = max([len(x) for x in sequences])
print(max_len)

33


In [19]:
padded_sequences = tf.keras.preprocessing.sequence.pad_sequences(
    sequences, padding='pre', maxlen= max_len
)

print(padded_sequences[0])

[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0 25 16]


In [20]:
padded_sequences.shape

(88279, 33)

## Task 5: Creating Training and Validation Sets

1. Creating training and validation sets

In [21]:
x, y = padded_sequences[:, :-1], padded_sequences[:, -1]
print(x.shape, y.shape)

(88279, 32) (88279,)


In [22]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y)

In [23]:
num_chars = len(char_to_index.keys()) + 1
num_chars

29

## Task 6: Creating the Model

In [24]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, MaxPool1D, LSTM, Bidirectional, Dense

In [25]:
model = Sequential( [
    Embedding(num_chars, 8, input_length = max_len-1),
    Conv1D (64, 5, strides=1, activation= 'tanh', padding= 'causal'),
    MaxPool1D (2),
    LSTM(32),
    Dense (num_chars, activation='softmax')
    ])
model.compile(
loss='sparse_categorical_crossentropy',
optimizer='adam',
metrics=['accuracy']
)
model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d (Conv1D)                      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d (MaxPooling1D)         │ ?                           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

## Task 7: Training the Model

In [33]:
h = model.fit(
x_train, y_train,
validation_data= (x_test, y_test),
epochs = 50, verbose = 2,
callbacks=[
tf.keras.callbacks. EarlyStopping (monitor= 'val_accuracy', patience=3)
])

Epoch 1/50
2070/2070 - 13s - 7ms/step - accuracy: 0.3976 - loss: 2.0041 - val_accuracy: 0.3503 - val_loss: 2.2090
Epoch 2/50
2070/2070 - 13s - 6ms/step - accuracy: 0.3986 - loss: 1.9991 - val_accuracy: 0.3520 - val_loss: 2.2073
Epoch 3/50
2070/2070 - 14s - 7ms/step - accuracy: 0.4001 - loss: 1.9958 - val_accuracy: 0.3507 - val_loss: 2.2063
Epoch 4/50
2070/2070 - 20s - 10ms/step - accuracy: 0.4018 - loss: 1.9918 - val_accuracy: 0.3519 - val_loss: 2.2085
Epoch 5/50
2070/2070 - 13s - 7ms/step - accuracy: 0.4043 - loss: 1.9870 - val_accuracy: 0.3536 - val_loss: 2.2067
Epoch 6/50
2070/2070 - 21s - 10ms/step - accuracy: 0.4039 - loss: 1.9828 - val_accuracy: 0.3529 - val_loss: 2.2098
Epoch 7/50
2070/2070 - 20s - 10ms/step - accuracy: 0.4056 - loss: 1.9795 - val_accuracy: 0.3529 - val_loss: 2.2096
Epoch 8/50
2070/2070 - 13s - 7ms/step - accuracy: 0.4066 - loss: 1.9759 - val_accuracy: 0.3553 - val_loss: 2.2107
Epoch 9/50
2070/2070 - 20s - 10ms/step - accuracy: 0.4092 - loss: 1.9719 - val_accura

## Task 8: Generate Names!

In [34]:
def generate_names(seed):
  for i in range(0, 40):
    seq = name_to_seq(seed)
    padded = tf.keras.preprocessing. sequence. pad_sequences ([seq], padding='pre', maxlen = max_len-1, truncating='pre')
    pred = model.predict(padded)[0]
    pred_char=index_to_char[tf.argmax (pred). numpy()]
    seed += pred_char
    if pred_char == '\t':
      break
    print(seed)

In [36]:
generate_names('s')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
sh
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
sha
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
shat
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
shath
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
shathe
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
shather
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
shather 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
shather s
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
shather st
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
shather str
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
shather stro
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
shather stron
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
shather strong
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
